In [1]:
import pprint
import os
import time
import random

import sacred
import pymongo
from sacred import Experiment
from sacred.observers import MongoObserver

## Create Experiment Object

In [2]:
ex_name='pythia_experiment'
db_name='sacred_demo'

# Check that MongoDB config is set
try:
    mongo_uri=os.environ['PYTHIA_MONGO_DB_URI']
except KeyError as e:
    print "Must define location of MongoDB in PYTHIA_MONGO_DB_URI for observer output"
    raise

# Don't create new experiment objects with the 
# same name in a single notebook session. This
# code only creates the ex object if it isn't already an Experiment
try:
    assert type(ex) is Experiment
except (NameError, AssertionError) as e:
    ex = Experiment(ex_name)
    ex.observers.append(MongoObserver.create(url=mongo_uri,
                                         db_name=db_name))

/home/pcallier/.local/lib/python3.4/site-packages/sacred-0.6.9-py3.4.egg/sacred/dependencies.py:210: UserWarning: Defining an experiment in interactive mode! The sourcecode cannot be stored and the experiment won't be reproducible
  warnings.warn("Defining an experiment in interactive mode! "


## Define Experiment

In [3]:
# Config blocks declare hyperparameters and default settings
@ex.config
def cfg():
    min_locs = 3
    dispersion_threshold = 50
    num_partitions = 30
    data_path = 'file:///data/*.json'
    
# Main blocks drive experiment logic
@ex.main
def run(min_locs, dispersion_threshold, num_partitions, data_path) :
    time.sleep(2)
    return [[1, 5, 6],[6,3,1]]

## Run Experiment

In [4]:
# Call .run() on an Experiment to run it, optionally allowing 
# its parameters to be set according to experiment defaults
ex.run()   # note that run parameters are magically filled in by cfg

INFO - pythia_experiment - Running command 'run'
INFO - pythia_experiment - Started
INFO - pythia_experiment - Result: [[1, 5, 6], [6, 3, 1]]
INFO - pythia_experiment - Completed after 0:00:02


## Check MongoDB

Shell out to mongo and get the runs database (run only if small); requires installation of mongodb client

%%bash

#sudo apt-get install -yq mongodb-clients

mongo --quiet "${PYTHIA_MONGO_DB_URI}/${db_name}" --eval 'var cursor = db.default.runs.find(); cursor.forEach(printjson)'